In [13]:
import pandas as pd, json, requests
from bs4 import BeautifulSoup


In [23]:
champion_stats_map = {}
champion_info_map = {}
with open("../data/championFull copy.json", "r") as f:
    data = json.load(f)
    
    for champ_name, champ_data in data['data'].items():
        attack = champ_data['info'][champ_name]['attack']
        defense = champ_data['info'][champ_name]['defense']
        magic = champ_data['info'][champ_name]['magic']
        difficulty = champ_data['info'][champ_name]['difficulty']
        
        
        champion_stats_map[champ_name] = champ_data['stats']
        champion_info_map[champ_name] = champ_data['info']

append elements of second dictionary into first dictionary, by key

In [34]:
print(champion_stats_map.keys())

dict_keys(['Aatrox', 'Ahri', 'Akali', 'Akshan', 'Alistar', 'Amumu', 'Anivia', 'Annie', 'Aphelios', 'Ashe', 'AurelionSol', 'Azir', 'Bard', 'Belveth', 'Blitzcrank', 'Brand', 'Braum', 'Briar', 'Caitlyn', 'Camille', 'Cassiopeia', 'Chogath', 'Corki', 'Darius', 'Diana', 'Draven', 'DrMundo', 'Ekko', 'Elise', 'Evelynn', 'Ezreal', 'Fiddlesticks', 'Fiora', 'Fizz', 'Galio', 'Gangplank', 'Garen', 'Gnar', 'Gragas', 'Graves', 'Gwen', 'Hecarim', 'Heimerdinger', 'Illaoi', 'Irelia', 'Ivern', 'Janna', 'JarvanIV', 'Jax', 'Jayce', 'Jhin', 'Jinx', 'Kaisa', 'Kalista', 'Karma', 'Karthus', 'Kassadin', 'Katarina', 'Kayle', 'Kayn', 'Kennen', 'Khazix', 'Kindred', 'Kled', 'KogMaw', 'KSante', 'Leblanc', 'LeeSin', 'Leona', 'Lillia', 'Lissandra', 'Lucian', 'Lulu', 'Lux', 'Malphite', 'Malzahar', 'Maokai', 'MasterYi', 'Milio', 'MissFortune', 'MonkeyKing', 'Mordekaiser', 'Morgana', 'Naafiri', 'Nami', 'Nasus', 'Nautilus', 'Neeko', 'Nidalee', 'Nilah', 'Nocturne', 'Nunu', 'Olaf', 'Orianna', 'Ornn', 'Pantheon', 'Poppy', 'P

In [36]:
for champ in champion_stats_map.items():
    # iterate through stats map and append using champ[key] = info
    

{'Janna': [{'hp': 570, 'hpperlevel': 90, 'mp': 360, 'mpperlevel': 50, 'movespeed': 325, 'armor': 28, 'armorperlevel': 4.5, 'spellblock': 30, 'spellblockperlevel': 1.3, 'attackrange': 550, 'hpregen': 5.5, 'hpregenperlevel': 0.55, 'mpregen': 11.5, 'mpregenperlevel': 0.4, 'crit': 0, 'critperlevel': 0, 'attackdamage': 47, 'attackdamageperlevel': 2.5, 'attackspeedperlevel': 3, 'attackspeed': 0.625}, {'attack': 3, 'defense': 5, 'magic': 7, 'difficulty': 7}], 'Kayle': [{'hp': 670, 'hpperlevel': 92, 'mp': 330, 'mpperlevel': 50, 'movespeed': 335, 'armor': 26, 'armorperlevel': 4.2, 'spellblock': 22, 'spellblockperlevel': 1.3, 'attackrange': 175, 'hpregen': 5, 'hpregenperlevel': 0.5, 'mpregen': 8, 'mpregenperlevel': 0.8, 'crit': 0, 'critperlevel': 0, 'attackdamage': 50, 'attackdamageperlevel': 2.5, 'attackspeedperlevel': 1.5, 'attackspeed': 0.625}, {'attack': 6, 'defense': 6, 'magic': 7, 'difficulty': 7}], 'Nami': [{'hp': 560, 'hpperlevel': 88, 'mp': 365, 'mpperlevel': 43, 'movespeed': 335, 'armo

In [37]:
merged_dict["Ziggs"]


[{'hp': 606,
  'hpperlevel': 106,
  'mp': 480,
  'mpperlevel': 23.5,
  'movespeed': 325,
  'armor': 18,
  'armorperlevel': 4.5,
  'spellblock': 30,
  'spellblockperlevel': 1.3,
  'attackrange': 550,
  'hpregen': 6.5,
  'hpregenperlevel': 0.6,
  'mpregen': 8,
  'mpregenperlevel': 0.8,
  'crit': 0,
  'critperlevel': 0,
  'attackdamage': 55,
  'attackdamageperlevel': 3.1,
  'attackspeedperlevel': 2,
  'attackspeed': 0.656},
 {'attack': 2, 'defense': 4, 'magic': 9, 'difficulty': 4}]

In [39]:
index = merged_dict.keys()
rows = [merged_dict[key] for key in index]

df = pd.DataFrame(rows, index=index)
df = df.fillna(0)
df.head()

,0,1
Janna,"{'hp': 570, 'hpperlevel': 90, 'mp': 360, 'mppe...","{'attack': 3, 'defense': 5, 'magic': 7, 'diffi..."
Kayle,"{'hp': 670, 'hpperlevel': 92, 'mp': 330, 'mppe...","{'attack': 6, 'defense': 6, 'magic': 7, 'diffi..."
Nami,"{'hp': 560, 'hpperlevel': 88, 'mp': 365, 'mppe...","{'attack': 4, 'defense': 3, 'magic': 7, 'diffi..."
Zac,"{'hp': 685, 'hpperlevel': 109, 'mp': 0, 'mpper...","{'attack': 3, 'defense': 7, 'magic': 7, 'diffi..."
Chogath,"{'hp': 644, 'hpperlevel': 94, 'mp': 270, 'mppe...","{'attack': 3, 'defense': 7, 'magic': 7, 'diffi..."


In [ ]:
page = requests.get("https://www.op.gg/statistics/champions?hl=en_US")
soup = BeautifulSoup(page.content, 'html.parser')

data = {}
for element in soup.select(".css-qzg52u.e13v5s1v10"):
    key = # some key based on the element
    value = element.text 
    data[key] = value

print(data) 

